In [1]:
from postgres_connect import *

In [2]:
path = 'tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)

Tunnel Started
Connection Made


In [3]:
query_path = 'rider_pip_closure.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
cursor = conn.cursor()
cursor.execute(custom_query)
conn.commit()
# conn.close()

In [4]:
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)

In [5]:
df

,key,pip_date,rider_id,shipping_city,rider_name,hub,vendor_name,rider_age,need_improvement,starting_fasr,closing_fasr,pip_status,pip_result,update_time
0,20240311-Manoj_18,2024-03-11,Manoj_18,Bangalore,Manoj,UUTR-Franchise,Blitz_Self,178,Morning Prepaid (80%) Morning Prepaid (76%) Mo...,70.0,74.0,closed,failed,2024-03-20 10:55:19.140886
1,20240311-Praveen_Kumar__18,2024-03-11,Praveen_Kumar__18,Bangalore,Praveen Kumar,UUTR-Franchise,Blitz_Self,172,Morning Prepaid (60%) Morning COD(44%),70.0,87.0,closed,passed,2024-03-20 10:55:19.140886
2,20240311-thejas_8892386333_032,2024-03-11,thejas_8892386333_032,Bangalore,thejas,CMRJ-Franchise,Hindushree,33,Morning Prepaid (57%),60.0,100.0,closed,passed,2024-03-20 10:55:19.140886
3,20240311-Aman_Sharma_11,2024-03-11,Aman_Sharma_11,Delhi,Aman Sharma,DEL FRH GKP,Hub Self,210,Morning Prepaid (80%) Morning Prepaid (54%) Mo...,57.0,64.0,closed,failed,2024-03-20 10:55:19.140886
4,20240311-Ashu__11,2024-03-11,Ashu__11,Delhi,Ashu,DEL FRH GKP,DUMMY_1,235,Morning COD(58%) Morning Prepaid (46%),69.0,77.0,closed,failed,2024-03-20 10:55:19.140886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,20240311-Madhu_9701731175_019,2024-03-11,Madhu_9701731175_019,Hyderabad,Madhu G,DS HYD SRNG,Hub Self,43,Morning Prepaid (77%),75.0,NaN,closed,failed,2024-03-20 10:55:19.140886
64,20240311-G_9652132045_019,2024-03-11,G_9652132045_019,Hyderabad,G Tharun,DS HYD SRNG,Hub Self,11,Morning Prepaid (78%),76.0,66.0,closed,failed,2024-03-20 10:55:19.140886
65,20240311-Akash_verma__3_1701592803,2024-03-11,Akash_verma__3_1701592803,Mumbai,Akash verma,DS BOM KRL,Ambika Logistics,91,Morning COD(43%),80.0,85.0,closed,passed,2024-03-20 10:55:19.140886
66,20240311-Shaikh_saeed_3_1701428480,2024-03-11,Shaikh_saeed_3_1701428480,Mumbai,Shaikh saeed,DS BOM KRL,Team HR,93,Morning Prepaid (80%),80.0,74.0,closed,failed,2024-03-20 10:55:19.140886


In [6]:
query_path = 'rider_pip_closure_performance.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
df_pip = pd.read_sql(custom_query,conn)
df_pip.head()

,pip_date,shipping_city,rider_id,rider_name,hub,performance_type,threshold,monday,tuesday,wednesday,thursday,friday,saturday,sunday,avg_fasr,status
0,2024-03-11,Bangalore,Bhaskar_R_2,Bhaskar R,CMRJ-Franchise,Morning-Prepaid,0.85,0.90,0.92,0.77,1.0,0.90,0.95,0.84,0.91,pass
1,2024-03-11,Bangalore,Bhaskar_R_2,Bhaskar R,CMRJ-Franchise,Morning-COD,0.60,1.00,0.66,0.66,0.9,0.50,0.77,0.72,0.70,pass
2,2024-03-11,Bangalore,Bhaskar_R_2,Bhaskar R,CMRJ-Franchise,Evening-Prepaid,0.80,0.85,0.75,NaN,NaN,0.33,NaN,0.66,0.71,fail
3,2024-03-11,Bangalore,Bhaskar_R_2,Bhaskar R,CMRJ-Franchise,Evening-COD,0.50,1.00,0.16,NaN,NaN,1.00,NaN,1.00,0.58,pass
4,2024-03-11,Bangalore,thejas_8892386333_032,thejas,CMRJ-Franchise,Morning-Prepaid,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail


In [7]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Filter failed riders
failed_riders = df[df['pip_result'] == 'failed']

# Group by city and hub
grouped_data = failed_riders.groupby(['shipping_city', 'hub']).size().reset_index(name='failed_count')

# Group by city to get total rider count
total_riders = df.groupby(['shipping_city', 'hub']).size().reset_index(name='total_count')

# Merge the two dataframes on city
result_df = pd.merge(total_riders, grouped_data, on=['shipping_city','hub'], how='left')

# Calculate the failed rate without decimals
result_df['failed_rate'] = (result_df['failed_count'] / result_df['total_count']) * 100
result_df['failed_rate'] = result_df['failed_rate'].fillna(0)
result_df['failed_rate'] = result_df['failed_rate'].astype(int)

# Email configuration
sender_email = "akshay@blitznow.in"
sender_password = "Outofblitz1!"

# Create the message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = "operations@blitznow.in"
message['Subject'] = f"Rider PIP Closure Report Week - {failed_riders['pip_date'].max()}"

# Create an Excel file with two sheets
excel_file_path = 'failed_riders_report.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Write the failed_riders DataFrame to the first sheet
    failed_riders.to_excel(writer, sheet_name='Failed_Riders_Data', index=False)

    # Write the df_pip DataFrame to the second sheet
    df_pip.to_excel(writer, sheet_name='Additional_Sheet', index=False)

# Attach the Excel file
with open(excel_file_path, 'rb') as excel_file:
    attachment = MIMEApplication(excel_file.read(), _subtype='xlsx')
    attachment.add_header('Content-Disposition', 'attachment', filename='pip_closure_report.xlsx')
    message.attach(attachment)

# Create an HTML table for the email body with formatted headers
html_table = result_df.to_html(index=False, border=1, classes='table', escape=False, justify='center')
html_table = html_table.replace('<th>', '<th style="font-weight:bold; background-color:lightblue; text-align:center;">').replace('_', ' ').upper()

# Add the table to the email body
body = f"""\
<html>
  <head></head>
  <body>
    <p>Dear recipients,</p>
    <p>Please find attached the report for RIDER PIP Result for the week - {failed_riders['pip_date'].max()}.</p>
    <p>Summary:</p>
    {html_table}
    <p>Best regards,<br>Your Name</p>
  </body>
</html>
"""

message.attach(MIMEText(body, 'html'))
recipients = "operations@blitznow.in"
# Connect to the SMTP server and send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipients, message.as_string())

print("Email sent successfully.")

Email sent successfully.


In [16]:
conn.close()